In [1]:
import pennylane as qml
from pennylane import numpy as np

def modexp_perm(m, modulus=15, base=2, nbits=4):
    return {a: (pow(base, m, modulus) * a) % modulus for a in range(2**nbits)}

def make_U_m(m, nbits=4):
    perm = modexp_perm(m, modulus=15, base=2, nbits=nbits)
    dim = 2**nbits
    U = np.zeros((dim, dim), dtype=complex)
    for a, b in perm.items():
        U[b, a] = 1
    return U

def compute_expected_index(result):
    bitstring = f"1{result:04b}00"
    return int(bitstring, 2)

def controlled_modular_multiplication(m, control_wire, target_wires):
    U = make_U_m(m, nbits=len(target_wires))
    qml.ControlledQubitUnitary(U, control_wires=[control_wire], wires=target_wires)

In [2]:
def verify_controlled_modular_multiplication(m):
    dev = qml.device("default.qubit", wires=7)  # 1 control + 4 target + 2 ancilla

    @qml.qnode(dev)
    def circuit():
        # prepare |1⟩_C ⊗ |1⟩_T ⊗ |00⟩_ancilla
        qml.PauliX(wires=0)
        qml.BasisState(np.array([0, 0, 0, 1]), wires=[1, 2, 3, 4])
        # apply controlled U^m
        controlled_modular_multiplication(
            m, control_wire=0, target_wires=[1, 2, 3, 4]
        )
        return qml.state()

    state = circuit()
    result = pow(2, m, 15)
    expected_index = compute_expected_index(result)
    probs = np.abs(state) ** 2
    got_index = int(np.argmax(probs))

    print(f"CU^{m} |1⟩|1⟩ → |1⟩|{result}⟩ (expected index {expected_index}, got {got_index})")
    print(f"Probability at expected index: {probs[expected_index]:.6f}")
    return got_index == expected_index

In [3]:
# Run the check for m=0,1,2,3
for m in range(4):
    assert verify_controlled_modular_multiplication(m)
print("All tests pass!")

CU^0 |1⟩|1⟩ → |1⟩|1⟩ (expected index 68, got 68)
Probability at expected index: 1.000000
CU^1 |1⟩|1⟩ → |1⟩|2⟩ (expected index 72, got 72)
Probability at expected index: 1.000000
CU^2 |1⟩|1⟩ → |1⟩|4⟩ (expected index 80, got 80)
Probability at expected index: 1.000000
CU^3 |1⟩|1⟩ → |1⟩|8⟩ (expected index 96, got 96)
Probability at expected index: 1.000000
All tests pass!
